#  Discrete Choice Dynamic Programming
## A Dynamic Probit Model

So far, we were working in *static* problems when the agent has to decide between alternatives that do not involve a dynamic nature.
In this lecture, we are going to cover a very simple example of Discrete Choice Dynamic Programming were time is another *state variable*. Again and for simplification, the decision space is minimal so we can focus on the innovation with respect to previous lecture: time.

Assume an agent $n \in N$ has to decide between two alternatives, working $(i=1)$ or leisure $(i=0)$. The *current* utility can be writen as

$U_{n1}=\beta x_n+\varepsilon_{n1} $ if the agent chose to work

$U_{n0}=\mu +\varepsilon_{n0} $ if the agent chose to do not work, i.e., leisure.

However, the agent cares about future. Future is uncertain and there is no perfect forseight, so the agent has to form an *expectation*. The maximization problem of the agent can be thought as the maximization of current utility plus expected utility from tomorrow to the last day. The value function of this agent, reads

$V(t,d;\varepsilon_t) =  max \{ U_{n1}-U_{n0} + EV_m(t+1,d;\varepsilon_{t+1}),EV_m(t+1,d+1;\varepsilon_{t+1})\}$

The utility of working can be written as a linear component plus an idiosyncratic shock. If an agent $n$ chooses to work, she achieves a utility that is a function of her education attainment $x$ plus a working-related shock. The parameter $\beta$ indicates the premium for each year of education, a parameter we are interested in recover it.

The utility of not-working can be written as a fixed component $\mu$ plus a leisure-related idiosyncratic shock. The fixed component can be interpreted as unemployment insurance and for simplicity, we are assuming it does not vary between agents. This assumption can be relaxed easily.

The shock $\varepsilon_{nj}$, $j=\{0,1\}$ is i.i.d. and choice specific, meaning that the agent face as many shocks as decisions she can choose from. In this model, we are going to assume that these idiosyncratic shocks come from standard Normal distrubution.

In [3]:
function probs(β::Float64,μ::Float64) # Probability of working
    Pr=fill(NaN, Tm, Tm); #Probability matrix for L=0, work
    ϵ_th=fill(NaN, Tm, Tm); # Threshold matrix for L=0, work
    V=fill(NaN, Tm, Tm); # Value funtion matrix
        for t=Tm:-1:1
            if t==Tm # Last day
                for d=0:Tm-1
                    ϵ_th[Tm,d+1]=-μ+P+β*(payment(d+1)-payment(d));
                    Pr[Tm,d+1]=cdf(dist,ϵ_th[Tm,d+1]);
                    V[Tm,d+1]=(1-Pr[Tm,d+1])*(μ-P+β*payment(d)+mean(trdist(ϵ_th[Tm,d+1],Inf)))+
                    Pr[Tm,d+1]*(β*payment(d+1))
                end
            else # From day 1 to Tm-1
                for d=0:t-1
                    ϵ_th[t,d+1]=-μ+P+V[t+1,d+2]-V[t+1,d+1]
                    Pr[t,d+1]=cdf(dist,ϵ_th[t,d+1])
                    V[t,d+1]=(1-Pr[t,d+1])*(μ-P+mean(trdist(ϵ_th[t,d+1],Inf))+V[t+1,d+1])+
                    Pr[t,d+1]*(V[t+1,d+2])
                end
            end
        end
    return Pr
end

probs (generic function with 1 method)